Assume an elastic rod of length $L$ and mass $m$. It is fixed at one side, and free to move at the other. Assume also a force $\vec{F}$ applied to the free end, orthogonal to the whisker.

<img src="images/elastic-whisker-deflection.png" alt="Drawing" style="width: 200px;"/>

Accodrding to Feynmann lecture notes, the approximate deflection of the rod 

$\Delta x = \alpha F (Ly^2 - y^3)$




According to
<a href="https://en.wikipedia.org/wiki/Euler%E2%80%93Bernoulli_beam_theory">Euler-Bernulli theory</a>,

the general equation for the elastic rod of uniform elasticity is given by

$EI \frac{\partial^4 x}{\partial y^4} + \mu \frac{\partial^2 x}{\partial t^2} = q(y,t)$

In case of a cantilever (elastic rod fixed at one end), the 
<a href="http://www.geom.uiuc.edu/education/calc-init/static-beam/support.html">boundary conditions</a> are given by

$\begin{matrix}
x(0,t) = 0\\
x'(0,t) = 0\\
x''(L,t) = 0\\
x'''(L,t) = F(t)
\end{matrix}$

where $F(t)$ is the force applied to the end of the rod, if at all.


Let us attempt to solve the equation using finite differences

$\partial_y^4 x(y,t) = \frac{1}{\Delta y^4} (x_{i-2, j} - 4 x_{i-1, j} + 6 x_{i,j} - 4 x_{i+1,j} + x_{i+2,j})$

$\partial_t^2 x(y,t) = \frac{1}{\Delta t^2} (x_{i, j-1} - 2 x_{i, j} + x_{i, j+1})$

Where $x_{i, j} = x(y_0 + i\Delta y, t_0 + j \Delta t)$. Thus the time-stepping algorithm can be written as

$x_{i, j+1} = 2x_{i,j} - x_{i,j-1} + \Delta t^2 (q_{i,j} + \frac{1}{\Delta y^4} (x_{i-2, j} - 4 x_{i-1, j} + 6 x_{i,j} - 4 x_{i+1,j} + x_{i+2,j}))$

Assuming the discretization of the rod into $n+1$ steps (namely, $i \in [0 .. n]$), we can also discretize the boundary conditions

$x_{0, j} = 0$

$x'_{0, j} = (x_{0, j} - x_{-1, j}) / \Delta t = 0$

$x''_{n, j} = (x_{n-1, j} - 2 x_{n, j} + x_{n+1, j}) / \Delta t^2 = 0$

$x'''_{n, j} = (-x_{n-1, j} + 3 x_{n, j} - 3 x_{n+1, j} + x_{n+2, j}) / \Delta t^3 = F_{n, j}$

We can now use the boundary conditions to rewrite the derivatives at the boundaries, thus eliminating terms that are outside of the domain

$x_{0, j} = 0$

$\partial_y^4 x_{1, j} = \frac{1}{\Delta y^4} (6 x_{1,j} - 4 x_{2,j} + x_{3,j})$

$\partial_y^4 x_{n-1, j} = \frac{1}{\Delta y^4} (x_{n-3, j} - 4 x_{n-2, j} + 5 x_{n-1,j} - 2 x_{n,j})$

$\partial_y^4 x_{n, j} = \frac{1}{\Delta y^4} (x_{n-2,j} - 2 x_{n-1,j} + x_{n,j} + F_{n,j} \Delta t^3)$

Finally, the updating stencil will be

$\vec{x}_{j+1} = 2 \vec{x}_j - \vec{x}_{j-1} + \Delta t^2 \vec{q}_j + \frac{\Delta t^5}{\Delta y^4} F_j \vec{e}_{n} +  \frac{\Delta t^2}{\Delta y^4} \times \vec{x}_j$

Where the elements of matrix $M$ are given by

$M_{i,j} = \delta_{i-2, j} - 4 \delta_{i-1, j} + 6 \delta_{i,j} - 4 \delta_{i+1,j} + \delta_{i+2,j}$

Except for the boundary rows

$M_{0,k} = [0, 0, ..., 0]$

$M_{1,k} = [0, 6, -4, 1, 0, 0, ..., 0]$

$M_{n-1,k} = [0, 0, ..., 0, 0, 1, -4, 5, -2]$

$M_{n,k} = [0, 0, ..., 0, 0, 1, -2, 1]$


In [ ]:
# 1) Assemble matrix 